In [19]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
from crewai import Agent, Crew, Task, LLM
import os
from dotenv import load_dotenv
load_dotenv()

# Mistral LLM Setup
llm = LLM(
    model="mistral/mistral-small-latest",
    api_key=os.getenv("MISTRAL_API_KEY")
)

## Role Playing, Focus and Cooperation

In [21]:
support_agent = Agent(
    role = 'Senior Support Representative',
    goal = "Be the most helpful and knowledgeable support agent in the team.",
    backstory= (
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
    ),
    instructions=(
        "You may ONLY use the tools provided. "
        "Do NOT attempt to call tools that are not listed in your toolset. "
        "There is NO 'add_image' tool. Focus on using 'ScrapeWebsiteTool' only. "
        "If unsure, explain it with text — do NOT invent tools."
    ),
    allow_delegation=False,
    verbose = True,
    llm = llm
)

- By default the allow_delegation has value as True which allows the agents to delegate work to the agent which is best suitated for the task.

In [22]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
    allow_delegation= True,
    llm = llm
)

## Tools, Guardrails and Memory

### Tools

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()  # Detect every link that it sees
scrape_tool = ScrapeWebsiteTool() # Detect only the particualr link

In [23]:
# in custom_tools.py
from crewai_tools import ScrapeWebsiteTool

In [24]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

### Creating Tasks
- You are passing the Tool on the Task Level.

In [25]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
        "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
        "You **MUST** use only the ScrapeWebsiteTool provided. "
        "Do NOT invent or use any tool that is not explicitly provided. "
        "There is NO 'add_image' tool available. "
        "You are expected to respond ONLY using valid tools or text output."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	# tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [26]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)

### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [27]:
crew = Crew(
    agents = [support_agent, support_quality_assurance_agent],
    tasks = [inquiry_resolution, quality_assurance_review],
    verbose = True,
    memory = False
)

In [28]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 42c21f4d-4ff3-49d6-a1b5-1d665dba8947                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Task: DeepLearningAI just reached out with a super important ask:                                              │
│  I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you   │
│  provide guidance?                                                                                              │
│                                                                                                                 │
│  Andrew Ng from DeepLearningAI is the one that reached out. You **MUST** use only the ScrapeWebsiteTool         │
│  provided. Do NOT invent or use any tool that is not explicitly provided. There is NO 'add_image' tool          │
│  available. You are expected to respond ONLY using valid tools or text output.                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Hello Andrew,                                                                                                  │
│                                                                                                                 │
│  Thank you for reaching out to CrewAI. I'm glad to help you set up your Crew and add memory to it. Here's a     │
│  step-by-step guide to help you:                                                                                │
│                                                                                                                 │
│  1. **Setting up a Crew:**                                                                                      │
│     - First, you need to define your Crew by specifying the roles and goals. You can do this by creating a      │
│  Python script or using our API. Here's a simple example:                                                       │
│                                                                                                                 │
│  ```python                                                                                                      │
│  from crewai import Crew, Agent, Task                                                                           │
│                                                                                                                 │
│  # Define your agents                                                                                           │
│  researcher = Agent(role="Senior Researcher", goal="Find cutting edge developments in AI")                      │
│  writer = Agent(role="Technical Writer", goal="Write a blog post about the research")                           │
│                                                                                                                 │
│  # Define your tasks                                                                                            │
│  task1 = Task(                                                                                                  │
│      description="Find the latest research on AI and summarize it",                                             │
│      expected_output="A bullet-point summary of the latest AI research",                                        │
│      agent=researcher,                                                                                          │
│      context={"current_date": "2023-10-01"},                                                                    │
│      async_execution=True                                                                                       │
│  )                                                                                                              │
│                                                                                                                 │
│  task2 = Task(                                                                                                  │
│      description="Write a blog post about the research",                                                        │
│      expected_output="A well-structured blog post",                                                             │
│      agent=writer,                                                                                              │
│      context={"tone": "professional"},                 

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0c5d8a9f-3af8-4b0e-9089-8aa8742f84fa                                                                     │
│  Agent: Senior Support Representative                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Hello Andrew,                                                                                                  │
│                                                                                                                 │
│  Thank you for reaching out to CrewAI. I'm delighted to assist you in setting up your Crew and adding memory    │
│  to it. Below is a comprehensive step-by-step guide to help you get started:                                    │
│                                                                                                                 │
│  ### 1. **Setting up a Crew:**                                                                                  │
│                                                                                                                 │
│  To define your Crew, you need to specify the roles and goals. This can be done by creating a Python script or  │
│  using our API. Here’s a detailed example:                                                                      │
│                                                                                                                 │
│  ```python                                                                                                      │
│  from crewai import Crew, Agent, Task, Process                                                                  │
│                                                                                                                 │
│  # Define your agents                                                                                           │
│  researcher = Agent(                                                                                            │
│      role="Senior Researcher",                                                                                  │
│      goal="Find cutting-edge developments in AI",                                                               │
│      backstory="A highly skilled researcher with a focus on AI advancements",                                   │
│      allow_delegation=False                                                                                     │
│  )                                                                                                              │
│                                                                                                                 │
│  writer = Agent(                                                                                                │
│      role="Technical Writer",                                                                                   │
│      goal="Write a blog post about the research",                                                               │
│      backstory="An experienced technical writer specializing in AI topics",                                     │
│      allow_delegation=False                                                                                     │
│  )                                                                                                              │
│                                                                                                                 │
│  # Define your tasks                                                                                            │
│  task1 = Task(                                                                                                  │
│      description="Find the latest research on AI and summarize it",                                             │
│      expected_output="A bullet-point summary of the latest AI research",                                        │
│      agent=researcher,                                

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Hello Andrew,                                                                                                  │
│                                                                                                                 │
│  Thank you for reaching out to CrewAI. I'm delighted to assist you in setting up your Crew and adding memory    │
│  to it. Below is a comprehensive step-by-step guide to help you get started:                                    │
│                                                                                                                 │
│  ### 1. **Setting up a Crew:**                                                                                  │
│                                                                                                                 │
│  To define your Crew, you need to specify the roles and goals. This can be done by creating a Python script or  │
│  using our API. Here’s a detailed example:                                                                      │
│                                                                                                                 │
│  ```python                                                                                                      │
│  from crewai import Crew, Agent, Task, Process                                                                  │
│                                                                                                                 │
│  # Define your agents                                                                                           │
│  researcher = Agent(                                                                                            │
│      role="Senior Researcher",                                                                                  │
│      goal="Find cutting-edge developments in AI",                                                               │
│      backstory="A highly skilled researcher with a focus on AI advancements",                                   │
│      allow_delegation=False                                                                                     │
│  )                                                                                                              │
│                                                                                                                 │
│  writer = Agent(                                                                                                │
│      role="Technical Writer",                                                                                   │
│      goal="Write a blog post about the research",                                                               │
│      backstory="An experienced technical writer specializing in AI topics",                                     │
│      allow_delegation=False                                                                                     │
│  )                                                                                                              │
│                                                                                                                 │
│  # Define your tasks                                                                                            │
│  task1 = Task(                                         

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7df5bea6-f538-4187-8e70-ed58a11cecae                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 42c21f4d-4ff3-49d6-a1b5-1d665dba8947                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Hello Andrew,                                                                                    │
│                                                                                                                 │
│  Thank you for reaching out to CrewAI. I'm delighted to assist you in setting up your Crew and adding memory    │
│  to it. Below is a comprehensive step-by-step guide to help you get started:                                    │
│                                                                                                                 │
│  ### 1. **Setting up a Crew:**                                                                                  │
│                                                                                                                 │
│  To define your Crew, you need to specify the roles and goals. This can be done by creating a Python script or  │
│  using our API. Here’s a detailed example:                                                                      │
│                                                                                                                 │
│  ```python                                                                                                      │
│  from crewai import Crew, Agent, Task, Process                                                                  │
│                                                                                                                 │
│  # Define your agents                                                                                           │
│  researcher = Agent(                                                                                            │
│      role="Senior Researcher",                                                                                  │
│      goal="Find cutting-edge developments in AI",                                                               │
│      backstory="A highly skilled researcher with a focus on AI advancements",                                   │
│      allow_delegation=False                                                                                     │
│  )                                                                                                              │
│                                                                                                                 │
│  writer = Agent(                                                                                                │
│      role="Technical Writer",                                                                                   │
│      goal="Write a blog post about the research",                                                               │
│      backstory="An experienced technical writer specializing in AI topics",                                     │
│      allow_delegation=False                                                                                     │
│  )                                                                                                              │
│                                                                                                                 │
│  # Define your tasks                                  

In [31]:
from IPython.display import Markdown
Markdown(result.raw)


Hello Andrew,

Thank you for reaching out to CrewAI. I'm delighted to assist you in setting up your Crew and adding memory to it. Below is a comprehensive step-by-step guide to help you get started:

### 1. **Setting up a Crew:**

To define your Crew, you need to specify the roles and goals. This can be done by creating a Python script or using our API. Here’s a detailed example:

```python
from crewai import Crew, Agent, Task, Process

# Define your agents
researcher = Agent(
    role="Senior Researcher",
    goal="Find cutting-edge developments in AI",
    backstory="A highly skilled researcher with a focus on AI advancements",
    allow_delegation=False
)

writer = Agent(
    role="Technical Writer",
    goal="Write a blog post about the research",
    backstory="An experienced technical writer specializing in AI topics",
    allow_delegation=False
)

# Define your tasks
task1 = Task(
    description="Find the latest research on AI and summarize it",
    expected_output="A bullet-point summary of the latest AI research",
    agent=researcher,
    context={"current_date": "2023-10-01"},
    async_execution=True
)

task2 = Task(
    description="Write a blog post about the research",
    expected_output="A well-structured blog post",
    agent=writer,
    context={"tone": "professional"},
    async_execution=True
)

# Create your crew
crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=2,
    process=Process.sequential  # or Process.parallel
)

# Kick off your crew
crew.kickoff()
```

### 2. **Adding Memory to your Crew:**

CrewAI supports memory through the use of context and tools like Langchain's memory tools. Here’s how you can add memory to your Crew:

```python
from langchain.memory import ConversationBufferMemory

# Define your agent with memory
researcher = Agent(
    role="Senior Researcher",
    goal="Find cutting-edge developments in AI",
    backstory="A highly skilled researcher with a focus on AI advancements",
    allow_delegation=False,
    memory=ConversationBufferMemory(memory_key="chat_history")
)

# Then, include this agent in your Crew as shown above
```

You can also use other types of memory like `ConversationSummaryMemory`, `ConversationBufferWindowMemory`, etc. For more information about these, please refer to the [Langchain documentation](https://langchain.readthedocs.io/en/latest/modules/memory.html).

### 3. **Kicking off your Crew:**

Once you've defined your Crew, you can kick it off by calling the `kickoff()` method as shown in the example above. This will start the execution of your Crew.

### Additional Resources:

- **CrewAI Documentation:** For more detailed information, please refer to our [official documentation](https://docs.crewai.com/).
- **Langchain Memory Documentation:** For a deeper understanding of memory tools, visit the [Langchain documentation](https://langchain.readthedocs.io/en/latest/modules/memory.html).

If you have any further questions or need more assistance, please don't hesitate to ask. We're here to help!

Best regards,
[Your Name]
Senior Support Representative, CrewAI